# Notebook 06: Putting It All Together

Congratulations on reaching the final notebook! Now we'll combine everything you've learned into a comprehensive technical analysis framework.

## What You'll Learn

1. Building a complete technical analysis dashboard
2. Combining multiple indicators effectively
3. Creating a systematic analysis framework
4. Analyzing UUE and other Malaysian stocks
5. Making informed trading decisions
6. Developing your own analysis checklist
7. Next steps in your technical analysis journey

## Time Required: 60-90 minutes

---

In [ ]:
# Setup
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings("ignore")

# Try importing pandas_ta
try:
    import pandas_ta as ta

    TA_AVAILABLE = True
    print("✅ pandas-ta imported successfully!")
except ImportError:
    TA_AVAILABLE = False
    print("⚠️ pandas-ta not found. Install it with:")
    print("   pip install pandas-ta")
    print("\nThis notebook requires pandas-ta for technical indicators.")


def analyze_stock(ticker, period="1y"):
    """
    Complete technical analysis of a stock
    """
    if not TA_AVAILABLE:
        print("\n⚠️ pandas-ta is required for this analysis.")
        print("Please install it with: pip install pandas-ta")
        return None

    print(f"\n{'='*60}")
    print(f"📊 Technical Analysis Dashboard: {ticker}")
    print(f"{'='*60}\n")

    try:
        # Fetch data
        stock = yf.Ticker(ticker)
        data = stock.history(period=period)

        if len(data) == 0:
            print(f"⚠️ No data returned for {ticker}")
            print("Try: 1155.KL (Maybank) or 1295.KL (Public Bank)")
            return None

        # Calculate all indicators
        # Moving Averages
        data["SMA_20"] = data["Close"].rolling(window=20).mean()
        data["SMA_50"] = data["Close"].rolling(window=50).mean()
        data["SMA_200"] = data["Close"].rolling(window=200).mean()
        data["EMA_12"] = data["Close"].ewm(span=12).mean()
        data["EMA_26"] = data["Close"].ewm(span=26).mean()

        # RSI
        data["RSI"] = ta.rsi(data["Close"], length=14)

        # MACD
        macd = ta.macd(data["Close"])
        data["MACD"] = macd["MACD_12_26_9"]
        data["MACD_signal"] = macd["MACDs_12_26_9"]
        data["MACD_hist"] = macd["MACDh_12_26_9"]

        # Volume
        data["Volume_MA"] = data["Volume"].rolling(window=20).mean()
        data["OBV"] = ta.obv(data["Close"], data["Volume"])

        # Get latest values
        latest = data.iloc[-1]

        # Print analysis
        print("📈 PRICE ANALYSIS")
        print(f"   Current Price: RM {latest['Close']:.2f}")
        print(f"   52-Week High: RM {data['High'].max():.2f}")
        print(f"   52-Week Low: RM {data['Low'].min():.2f}")

        print("\n🎯 TREND ANALYSIS (Moving Averages)")
        print(
            f"   SMA 20: RM {latest['SMA_20']:.2f} {'✅ Above' if latest['Close'] > latest['SMA_20'] else '⚠️ Below'}"
        )
        print(
            f"   SMA 50: RM {latest['SMA_50']:.2f} {'✅ Above' if latest['Close'] > latest['SMA_50'] else '⚠️ Below'}"
        )
        if not pd.isna(latest["SMA_200"]):
            print(
                f"   SMA 200: RM {latest['SMA_200']:.2f} {'✅ Above' if latest['Close'] > latest['SMA_200'] else '⚠️ Below'}"
            )

        print("\n💪 MOMENTUM INDICATORS")
        print(f"   RSI: {latest['RSI']:.1f}", end=" ")
        if latest["RSI"] > 70:
            print("⚠️ OVERBOUGHT")
        elif latest["RSI"] < 30:
            print("✅ OVERSOLD")
        else:
            print("📊 NEUTRAL")

        print(f"   MACD: {latest['MACD']:.4f}")
        print(
            f"   Signal: {latest['MACD_signal']:.4f} {'📈 Bullish' if latest['MACD'] > latest['MACD_signal'] else '📉 Bearish'}"
        )

        print("\n📊 VOLUME ANALYSIS")
        volume_ratio = latest["Volume"] / latest["Volume_MA"]
        print(f"   Today's Volume: {latest['Volume']:,.0f}")
        print(f"   20-Day Avg: {latest['Volume_MA']:,.0f}")
        print(f"   Ratio: {volume_ratio:.2f}x", end=" ")
        if volume_ratio > 1.5:
            print("🔥 HIGH")
        elif volume_ratio < 0.5:
            print("😴 LOW")
        else:
            print("📊 NORMAL")

        # Overall signal
        print("\n" + "=" * 60)
        print("🎯 OVERALL SIGNAL")

        bullish_signals = 0
        bearish_signals = 0

        if latest["Close"] > latest["SMA_50"]:
            bullish_signals += 1
        else:
            bearish_signals += 1

        if latest["RSI"] < 40:
            bullish_signals += 1
        elif latest["RSI"] > 60:
            bearish_signals += 1

        if latest["MACD"] > latest["MACD_signal"]:
            bullish_signals += 1
        else:
            bearish_signals += 1

        print(f"   Bullish Signals: {bullish_signals}/3")
        print(f"   Bearish Signals: {bearish_signals}/3")

        if bullish_signals > bearish_signals:
            print("\n   ✅ OVERALL: BULLISH BIAS")
        elif bearish_signals > bullish_signals:
            print("\n   ⚠️ OVERALL: BEARISH BIAS")
        else:
            print("\n   📊 OVERALL: NEUTRAL")

        print("=" * 60 + "\n")

        return data

    except Exception as e:
        print(f"⚠️ Error during analysis: {e}")
        return None


# Example: Analyze UUE
print("Analyzing 0310.KL (UUE Holdings Berhad)...")
uue_data = analyze_stock("0310.KL")

In [ ]:
# Create comprehensive visualization
def create_dashboard(data, ticker):
    """
    Create a 4-panel technical analysis dashboard
    """
    if data is None or len(data) == 0:
        print("⚠️ No data available for dashboard.")
        return

    if "SMA_20" not in data.columns:
        print("⚠️ Technical indicators not calculated.")
        return

    fig = make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        row_heights=[0.4, 0.2, 0.2, 0.2],
        subplot_titles=(f"{ticker} Price with Moving Averages", "RSI (14)", "MACD", "Volume"),
    )

    # Panel 1: Price with MAs
    fig.add_trace(
        go.Scatter(x=data.index, y=data["Close"], name="Price", line=dict(color="blue", width=2)),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data.index, y=data["SMA_20"], name="SMA 20", line=dict(color="green", dash="dot")
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data.index, y=data["SMA_50"], name="SMA 50", line=dict(color="orange", dash="dash")
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(x=data.index, y=data["SMA_200"], name="SMA 200", line=dict(color="red")),
        row=1,
        col=1,
    )

    # Panel 2: RSI
    if "RSI" in data.columns:
        fig.add_trace(
            go.Scatter(x=data.index, y=data["RSI"], name="RSI", line=dict(color="purple")),
            row=2,
            col=1,
        )
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
        fig.add_hline(y=50, line_dash="dot", line_color="gray", row=2, col=1)

    # Panel 3: MACD
    if "MACD" in data.columns:
        fig.add_trace(
            go.Scatter(x=data.index, y=data["MACD"], name="MACD", line=dict(color="blue")),
            row=3,
            col=1,
        )
        fig.add_trace(
            go.Scatter(
                x=data.index, y=data["MACD_signal"], name="Signal", line=dict(color="orange")
            ),
            row=3,
            col=1,
        )
        colors = ["green" if val >= 0 else "red" for val in data["MACD_hist"]]
        fig.add_trace(
            go.Bar(
                x=data.index,
                y=data["MACD_hist"],
                name="Histogram",
                marker_color=colors,
                showlegend=False,
            ),
            row=3,
            col=1,
        )

    # Panel 4: Volume
    colors = [
        "green" if data["Close"].iloc[i] >= data["Open"].iloc[i] else "red"
        for i in range(len(data))
    ]
    fig.add_trace(
        go.Bar(
            x=data.index, y=data["Volume"], name="Volume", marker_color=colors, showlegend=False
        ),
        row=4,
        col=1,
    )
    if "Volume_MA" in data.columns:
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data["Volume_MA"],
                name="Vol MA",
                line=dict(color="orange", dash="dash"),
            ),
            row=4,
            col=1,
        )

    # Update layout
    fig.update_layout(height=1200, template="plotly_white", hovermode="x unified", showlegend=True)

    fig.update_yaxes(title_text="Price (RM)", row=1, col=1)
    fig.update_yaxes(title_text="RSI", row=2, col=1)
    fig.update_yaxes(title_text="MACD", row=3, col=1)
    fig.update_yaxes(title_text="Volume", row=4, col=1)
    fig.update_xaxes(title_text="Date", row=4, col=1)

    fig.show()
    print("✅ Complete technical analysis dashboard created!")


# Create dashboard for UUE
if uue_data is not None:
    create_dashboard(uue_data, "0310.KL - UUE Holdings Berhad")
else:
    print("⚠️ Cannot create dashboard - no data available.")

In [ ]:
# Compare multiple stocks
if not TA_AVAILABLE:
    print("⚠️ pandas-ta is required for multi-stock analysis.")
    print("Install it with: pip install pandas-ta")
else:
    stocks_to_analyze = [
        ("0310.KL", "UUE Holdings Berhad"),  # Construction/Engineering
        ("1155.KL", "Maybank"),
        ("1295.KL", "Public Bank"),
        ("5296.KL", "Tenaga Nasional"),
    ]

    print("\n" + "=" * 60)
    print("📊 COMPARATIVE ANALYSIS OF MALAYSIAN STOCKS")
    print("=" * 60)

    comparison_results = []

    for ticker, name in stocks_to_analyze:
        try:
            data = analyze_stock(ticker, period="3mo")
            if data is not None:
                comparison_results.append((ticker, name, data))
        except Exception as e:
            print(f"⚠️ Could not analyze {ticker}: {e}")

    if len(comparison_results) > 0:
        print("\n✅ Comparative analysis complete!")
    else:
        print("\n⚠️ No stocks could be analyzed. Check your internet connection.")

## Part 2: Your Technical Analysis Checklist

Use this checklist before making any trading decision:

### ✅ Analysis Checklist

**1. Trend Analysis**
- [ ] What's the long-term trend (SMA 200)?
- [ ] What's the medium-term trend (SMA 50)?
- [ ] What's the short-term trend (SMA 20)?
- [ ] Are trends aligned or conflicting?

**2. Support & Resistance**
- [ ] Where are the key support levels?
- [ ] Where are the key resistance levels?
- [ ] Is price near a major level?
- [ ] Has price broken through any levels?

**3. Momentum**
- [ ] What's the RSI reading? (Overbought/Oversold?)
- [ ] Is MACD bullish or bearish?
- [ ] Are there any divergences?

**4. Volume**
- [ ] Is volume confirming the price move?
- [ ] Is OBV trending with price?
- [ ] Any unusual volume spikes?

**5. Overall Signal**
- [ ] Do multiple indicators agree?
- [ ] Is the risk/reward favorable?
- [ ] Where will you place stop loss?
- [ ] What's your target price?

## Part 3: Analyzing Multiple Malaysian Stocks

Let's compare UUE with other popular Malaysian stocks

## Part 4: Next Steps in Your Journey

Congratulations on completing this technical analysis course! 🎉

### What You've Accomplished

✅ Understand technical analysis fundamentals
✅ Read candlestick charts
✅ Use moving averages for trend analysis
✅ Apply RSI and MACD for momentum
✅ Analyze volume patterns
✅ Build comprehensive analysis dashboards

### Your Next Steps

1. **Practice Daily**
   - Analyze UUE and other stocks daily
   - Track your analysis in a journal
   - Learn from mistakes

2. **Paper Trading**
   - Practice with virtual money first
   - Test your strategies without risk
   - Build confidence

3. **Learn More Advanced Topics**
   - Fibonacci retracements
   - Ichimoku Cloud
   - Elliott Wave Theory
   - Advanced chart patterns

4. **Combine with Fundamental Analysis**
   - Learn to read financial statements
   - Understand PE ratios, EPS, ROE
   - Use both technical and fundamental

5. **Join Communities**
   - i3investor forum
   - Malaysian stock trading groups
   - Learn from experienced traders

6. **Continue Learning**
   - Check out the KLSE Backtesting project
   - Read books on technical analysis
   - Watch educational videos

### Important Reminders

⚠️ **Never invest more than you can afford to lose**
⚠️ **Always use stop losses**
⚠️ **Don't rely on one indicator alone**
⚠️ **Manage your emotions**
⚠️ **Keep learning and improving**

### Resources

**Books**:
- "Technical Analysis of the Financial Markets" - John Murphy
- "Japanese Candlestick Charting Techniques" - Steve Nison

**Websites**:
- [Investopedia](https://www.investopedia.com/)
- [TradingView](https://www.tradingview.com/)
- [i3investor](https://klse.i3investor.com/)

**Other Projects in This Portfolio**:
- KLSE Stock Screener (for finding stocks)
- KLSE Backtesting (for testing strategies)

---

## 🎓 Final Thoughts

Technical analysis is both an art and a science. It takes time to master, but with practice and discipline, you can become proficient.

Remember:
- **Knowledge + Discipline + Patience = Success**
- Start small and learn continuously
- Protect your capital first, profits second
- The market will always be there tomorrow

**Thank you for completing this journey!**

Feel free to revisit these notebooks anytime and continue practicing with UUE and other Malaysian stocks.

---

**Happy Trading! 📈💰🎯**

*"The stock market is filled with individuals who know the price of everything, but the value of nothing." - Philip Fisher*